In [ ]:
try:
    import sim
except:
    print ('--------------------------------------------------------------')
    print ('"sim.py" could not be imported. This means very probably that')
    print ('either "sim.py" or the remoteApi library could not be found.')
    print ('Make sure both are in the same folder as this file,')
    print ('or appropriately adjust the file "sim.py"')
    print ('--------------------------------------------------------------')
    print ('')

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math

'''
readSensorData - It will try to capture the range and angle data from the simulator.
                 The request for the range data is sent in streaming mode to force
                 it to sync with the angle data request which acts as a mutex.

inputs:
    -clientId: simulator client id obtained through a successfull connection with the simulator.
    -range_data_signal_id: string containing the range data signal pipe name.
    -angle_data_signal_id: string containing the angle data signal pipe name.
outputs:
    -returns None if no data is recovered.
    -returns two arrays, one with data range and the other with their angles, if data was 
    retrieved successfully.
'''
def readSensorData(clientId=-1, 
                    range_data_signal_id="hokuyo_range_data", 
                    angle_data_signal_id="hokuyo_angle_data"):

    # the first call should be non-blocking to avoid getting out-of-sync angle data
    returnCodeRanges, string_range_data = sim.simxGetStringSignal(clientId, range_data_signal_id, sim.simx_opmode_streaming)

    # the second call should block to avoid out-of-sync scenarios
    # between your python script and the simulator's main loop
    # (your script may be slower than the simulator's main loop, thus
    # slowing down data processing)
    returnCodeAngles, string_angle_data = sim.simxGetStringSignal(clientId, angle_data_signal_id, sim.simx_opmode_blocking)

    # check the if both data were obtained correctly
    if returnCodeRanges == 0 and returnCodeAngles == 0:
        # unpack data from range and sensor messages
        raw_range_data = sim.simxUnpackFloats(string_range_data)
        raw_angle_data = sim.simxUnpackFloats(string_angle_data)

        return raw_range_data, raw_angle_data

    # return none in case were nothing was gotten from the simulator
    return None

def Rz(theta):
  
    return np.array([[ np.cos(theta), -np.sin(theta), 0 ],
                      [ np.sin(theta), np.cos(theta) , 0 ],
                      [ 0            , 0             , 1 ]])

def att_force(q, goal, katt=.2):
    return katt*(goal - q)

def rep_force(q, hokuyo, laser_data, max_sensor_range=5, R=2, krep=.01):
    repAdd = np.arange(2).reshape(1,2)
    d = 1
    
    rotationWL = Rz(hokuyo[3])
    translationWL = [hokuyo[0], hokuyo[1], hokuyo[2]]
    transformWL = np.column_stack((rotationWL, translationWL))
    aux = np.array([0, 0, 0, 1])
    transformWL = np.row_stack((transformWL, aux))

    for i in range(len(laser_data)):
        ang, dist = laser_data[i]
        if (max_sensor_range - dist) > 0.1:
            x = dist * np.cos(ang)
            y = dist * np.sin(ang)
            
            objCoordsT = transformWL @ np.array([x, y, 0, 1])

            v = q[:2] - [objCoordsT[0], objCoordsT[1]]
            d = np.linalg.norm(v, axis=0)
            d = d.reshape((len(v)-1, 1))
            rep = (1/d**2)*((1/d)-(1/R))*(v/d)
            rep_x = rep[:, 0]
            rep_y = rep[:, 1]
            
            np.add(repAdd, rep, out=repAdd, casting='unsafe')

#     invalid = np.squeeze(d > R)
#     repAdd[invalid, :] = 0
    
    return krep*repAdd


In [48]:
####################################################################################
#                                                                                  #
#                 LEMBRE-SE QUE A SIMULAÇÃO DEVE ESTAR EM EXECUÇÃO!                #
#                                                                                  #
####################################################################################

print ('Program started')
sim.simxFinish(-1) # just in case, close all opened connections
clientID=sim.simxStart('127.0.0.1',19999,True,True,5000,5) # Connect to CoppeliaSim

if clientID!=-1:
    print ('Connected to remote API server')

    robotName = 'Pioneer_p3dx'
    hokuyoName = 'fastHokuyo'
    returnCode, robotHandle = sim.simxGetObjectHandle(clientID, robotName, sim.simx_opmode_oneshot_wait)
    returnCode, hokuyoHandle = sim.simxGetObjectHandle(clientID, hokuyoName, sim.simx_opmode_oneshot_wait)
    
    returnCode, robotLeftMotorHandle  = sim.simxGetObjectHandle(clientID, robotName + '_leftMotor', sim.simx_opmode_oneshot_wait)
    returnCode, robotRightMotorHandle = sim.simxGetObjectHandle(clientID, robotName + '_rightMotor', sim.simx_opmode_oneshot_wait)
    
    returnCode, robotPos = sim.simxGetObjectPosition(clientID, robotHandle, -1, sim.simx_opmode_oneshot_wait)
    returnCode, robotOri = sim.simxGetObjectOrientation(clientID, robotHandle, -1, sim.simx_opmode_oneshot_wait)
    
    # Posição de teste para cena Teste1
#     pgoal = np.array([4, 8])
    
    # Segunda posição de teste para cena Teste2
    pgoal = np.array([2, 7])
    
    # Handle para os dados do LASER
    laser_range_data = "hokuyo_range_data"
    laser_angle_data = "hokuyo_angle_data"
    
    returnCode = 1
    while returnCode != 0:
        returnCode, range_data = sim.simxGetStringSignal(clientID, laser_range_data, sim.simx_opmode_streaming + 10)

    raw_range_data, raw_angle_data = readSensorData(clientID, laser_range_data, laser_angle_data)
    laser_data = np.array([raw_angle_data, raw_range_data]).T

    L = 0.381
    r = 0.0975
    maxv = 1.0
    maxw = np.deg2rad(45)
    
    rho = np.inf
    while rho > .05:
        
        returnCode, robotPos = sim.simxGetObjectPosition(clientID, robotHandle, -1, sim.simx_opmode_oneshot_wait)
        returnCode, robotOri = sim.simxGetObjectOrientation(clientID, robotHandle, -1, sim.simx_opmode_oneshot_wait)        
        robotConfig = np.array([robotPos[0], robotPos[1], robotOri[2]])
    
        returnCode, hokuyoPos = sim.simxGetObjectPosition(clientID, hokuyoHandle, -1, sim.simx_opmode_oneshot_wait)
        returnCode, hokuyoOri = sim.simxGetObjectOrientation(clientID, hokuyoHandle, -1, sim.simx_opmode_oneshot_wait)
        hokuyoConfig = np.array([hokuyoPos[0], hokuyoPos[1], hokuyoPos[2], hokuyoOri[2]])
        
        # Gets attractive force
        Fatt = att_force(robotConfig[:2], pgoal)
        Fatt_x = Fatt[0]
        Fatt_y = Fatt[1]
        
        raw_range_data, raw_angle_data = readSensorData(clientID, laser_range_data, laser_angle_data)
        laser_data = np.array([raw_angle_data, raw_range_data]).T
        
        # Gets Repulsive force from laser data and adds to attraction force
        Frep = rep_force(robotConfig, hokuyoConfig, laser_data)
        Ft = Fatt + Frep

        Ft_x = Ft[:, 0]
        Ft_y = Ft[:, 1]
        
        rho = np.sqrt(Ft_x**2 + Ft_y**2)
                
        kr = 1
        kt = 2
        
        v = kr*(Ft_x*np.cos(robotConfig[2]) + Ft_y*np.sin(robotConfig[2]))
        w = kt*(np.arctan2(Ft_y, Ft_x) - robotConfig[2])
                
        v = max(min(v, maxv), -maxv)
        w = max(min(w, maxw), -maxw)        
        
        vr = ((2.0*v) + (w*L))/(2.0*r)
        vl = ((2.0*v) - (w*L))/(2.0*r)
        
        sim.simxSetJointTargetVelocity(clientID, robotRightMotorHandle, vr, sim.simx_opmode_oneshot_wait)
        sim.simxSetJointTargetVelocity(clientID, robotLeftMotorHandle, vl, sim.simx_opmode_oneshot_wait)

    sim.simxSetJointTargetVelocity(clientID, robotRightMotorHandle, 0, sim.simx_opmode_oneshot_wait)
    sim.simxSetJointTargetVelocity(clientID, robotLeftMotorHandle, 0, sim.simx_opmode_oneshot_wait)

    # Now close the connection to CoppeliaSim:
    sim.simxFinish(clientID)
    
else:
    print ('Failed connecting to remote API server')
    
print ('Program ended')

Program started
Connected to remote API server
Program ended
